In [1]:
import tensorflow as tf
import tensorflow_hub as hub
print(tf.__version__)

elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)


In [4]:
# default repre
embeddings = elmo(['Emily ate a lot of pi'], signature="default", as_dict=True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
embeddings

{'lstm_outputs2': <tf.Tensor 'module_apply_default_1/concat_1:0' shape=(1, ?, 1024) dtype=float32>,
 'sequence_len': <tf.Tensor 'module_apply_default_1/Sum:0' shape=(1,) dtype=int32>,
 'lstm_outputs1': <tf.Tensor 'module_apply_default_1/concat:0' shape=(1, 6, 1024) dtype=float32>,
 'elmo': <tf.Tensor 'module_apply_default_1/aggregation/mul_3:0' shape=(1, 6, 1024) dtype=float32>,
 'default': <tf.Tensor 'module_apply_default_1/truediv:0' shape=(1, 1024) dtype=float32>,
 'word_emb': <tf.Tensor 'module_apply_default_1/bilm/Reshape_1:0' shape=(1, 6, 512) dtype=float32>}

In [6]:
embeddings['default']

<tf.Tensor 'module_apply_default_1/truediv:0' shape=(1, 1024) dtype=float32>

In [ ]:
def embed_p(corpusdir="../../", 
            corpusfile="NYTcorpus_updated.p", 
            logfile="log.txt", 
            outfile="embedded_100%.p", 
            labelfile="binarylabels_100%.p", 
            labeldictfile="labelsdict_100%.p"):
    # embed each article in a pickle format corpus into a singular vector using vocabfile as the vocabulary
    # the format of the corpus is assumed to be the same as NYTcorpus.p

    f = open(logfile, 'a+')
    f.write("DOC2VEC_EMBED_P\n")
    try:
        # load the whole corpus
        with open(corpusdir + corpusfile, "rb") as corpus:
            all_data = pickle.load(corpus)
            f.write("Loaded corpus!\n")
            data = []
            labels = []
            l = len(all_data)
            for i in range(0, l): # memory errors rip
                article = all_data[i][2]
                # tokenize each article
                data.append(TaggedDocument(list(tokenize(article)), [i])) # article text
                labels.append(all_data[i][3:]) # article labels
            label_matrix, label_dict = transform_labels(labels) # make labels binary
        f.write("Words are tokenized\n")

        # transform data by doc2vec
        model = Doc2Vec(data, vector_size=100, window=5, workers=12, epochs=10)
        model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
        vec_data = []
        for i in range(0, l):
            vec_data.append(model.docvecs[i])

        # transform the test data by tf_idf as well
        f.write("Doc2Vec done\n")

        # save the results into pickle files
        with open(outfile, "wb") as out:
            pickle.dump(vec_data, out)
            f.write("Dumped data output at " + outfile + "\n")
        
        with open(labelfile, "wb") as label:
            pickle.dump(label_matrix, label)
            f.write("Dumped label output at " + labelfile + "\n")

        with open(labeldictfile, "wb") as labeldict:
            pickle.dump(label_dict, labeldict)
            f.write("Dumped label dictionary at " + labeldictfile + "\n")
        
    except Exception:
        traceback.print_exc(file=f)

    f.close()